In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

In [2]:
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
wikipairs = wiki.flatMap(lambda x: [((x[i],x[i+1]),1) for i in range(0,len(x)-1)]).reduceByKey(lambda x,y:x+y)
wikinarod = wikipairs.filter(lambda x: x[0][0].lower() == "narodnaya").sortBy(lambda a: a[0][1])

In [3]:
result = wikinarod.collect()

In [22]:
for word in result:
    print "%s_%s\t%d" % (word[0][0].lower(),word[0][1].lower(),word[1])

narodnaya_gazeta	1
narodnaya_volya	9
